In [91]:
import petl as etl
from targetprocess.api import TargetProcessAPIClient
from targetprocess.serializers import TargetProcessSerializer

In [92]:
tp = TargetProcessAPIClient(api_url='https://name.tpondemand.com/api/v1/', user='user@site.com', password='123')

In [93]:
request_json = tp.get_collection(collection="Release", take = 1000,include="[Name,StartDate,EndDate,Project]" )
ListJson = TargetProcessSerializer.deserialize(request_json)  

In [94]:
Rtable1 = etl.fromdicts(ListJson, header=['Name','StartDate', 'EndDate','Project'])
Rtable2 = etl.addfield(Rtable1, 'Type', lambda row: row['Project']['Name'])
Rtable3 = etl.cutout(Rtable2, 'Project')
Rtable4 = etl.select(Rtable3,lambda rec: rec.Type != 'Flipbox' and rec.Name != 'Priority Backlog') 

In [95]:
RList = etl.dicts(Rtable4)
ReleaseList = list(RList)

In [96]:
request_json2 = tp.get_collection(collection="Requests", take = 10000000,include="[Id,Name,RequestType,CreateDate]")
ListJson2 = TargetProcessSerializer.deserialize(request_json2)

In [97]:
Rqtable1 = etl.fromdicts(ListJson2, header=['Id','CreateDate', 'RequestType', 'Name'])
Rqtable2 = etl.addfield(Rqtable1, 'Type', lambda row: row['RequestType']['Name'])
Rqtable3 = etl.cut(Rqtable2, 'Id', 'CreateDate','Name','Type')
Rqtable4 = etl.select(Rqtable3,lambda rec: rec.Type == 'Ошибка')

In [98]:
table5 = etl.addfield(Rqtable4, 'Release', lambda row: [ r['Name'] for r in ReleaseList if (r['StartDate']<row['CreateDate']<r['EndDate'])])


In [99]:
table6 = etl.convert(table5,'Release', lambda v:v[0] if v==v else '' )


In [100]:
etl.toxlsx(table6,'BugListWithRelease.xlsx')